In [1]:
import sys, os
sys.path.append(os.getcwd() + '/../')
from itergen.main import IterGen
from itergen import Dataset
from itergen.syncode.syncode.evaluation.sql_eval import SQLEval

json_dataset = Dataset('spider', language='json')
iter_gen = IterGen(grammar='sql', model_id="meta-llama/Llama-2-7b-chat-hf", parse_output_only=True, do_sample=True, temperature=0.5, stop_strings=['\n\n'], max_new_tokens=100)

/home/shubham/anaconda3/envs/codex/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.24it/s]


In [ ]:

def exists_column(schema, column_name):
        return True
    
def exists_table(schema, table_name):
        return True

def generate_sql(self, problem):
    iter_gen = IterGen(grammar='sql', model_id="meta-llama/Llama-2-7b-chat-hf", parse_output_only=True, do_sample=True, temperature=0.5)

    iter_gen.start(problem['prompt'])
    schema = problem['schema']
    
    while not iter_gen.finished():
        out = iter_gen.forward('name', num=1)

        last_column = iter_gen.view('column_name')[-1]         
        if not exists_column(schema, last_column):
            iter_gen.backward('column_name')
            continue

        last_table = iter_gen.view('table_name')[-1]
        if not exists_table(schema, last_table):
            iter_gen.backward('table_name')
            continue     
        print(out)
    return out

In [10]:
from tqdm import tqdm

samples = []
tmp_res_file = '/tmp/sql_output.txt'
f = open(tmp_res_file, 'w')
problems = json_dataset.problems[:100]
pbar = tqdm(total=len(problems))


for task_id, problem in enumerate(problems):
    iter_gen.start(problem['prompt'])
    out = iter_gen.forward()

    raw_completion = out[0]
    completion = json_dataset.post_process_answer(raw_completion)
    res = dict(
        task_id=task_id,
        completion=completion,
    )
    samples += [res]
    f.write(completion + '\n')
    pbar.update(1)

pbar.close()
f.close()

SQLEval.compute_accuracy(samples, tmp_res_file)

  9%|▉         | 9/100 [01:52<18:59, 12.52s/it]

KeyError: (__ANON_4, 3)